In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH02-Prompt")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH02-Prompt


LLM 객체를 정의합니다.

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

### 방법 1. from_template() 메서드를 사용하여 PromptTemplate 객체 생성
- 치환될 변수 `{ 변수 }`로 묶어서 템플릿을 정의합니다.

In [4]:
from langchain_core.prompts import PromptTemplate

# template 정의. {country}는 변수로, 이후에 값이 들어갈 자리를 의미
template = "{country}의 수도는 어디인가요?"

# from_teplate 메서드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

`country`변수에 값을 넣어서 문장을 생성할 수 있습니다.

In [5]:
# prompt 생성. format 메서드를 이용하여 변수에 값을 넣어준다.
prompt = prompt.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [6]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메서드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

# chain 생성
chain = prompt | llm

In [8]:
# country 변수에 입력된 값이 자동으로 치환되어 수행됨
chain.invoke({"country" : "대한민국"}).content

'대한민국의 수도는 서울입니다.'

### 방법 2. PromptTemplate 객체 생성과 동시에 prompt 생성
추가 유효성 검사를 위해 `input_variables`를 명시적으로 지정하세요. </br>

이러한 변수는 인스턴스화 중에 템플릿 문자열에 있는 변수와 비교하여 불일치하는 경우 예외를 발생시킵니다.

In [9]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country"],
)

prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [10]:
# prompt 생성
prompt.format(country="대한민국")

'대한민국의 수도는 어디인가요?'

In [11]:
# template 정의
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2" : "미국" # dictionary 형태로 partial_variables를 전달
    },
)

prompt

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [12]:
prompt.format(country1="대한민국")

'대한민국과 미국의 수도는 각각 어디인가요?'

In [13]:
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country1', 'country2'], input_types={}, partial_variables={}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [15]:
prompt_partial = prompt.partial(country2="캐나다")
prompt_partial

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [16]:
prompt_partial.format(country1="대한민국")

'대한민국과 캐나다의 수도는 각각 어디인가요?'

In [17]:
chain = prompt_partial | llm

In [18]:
chain.invoke("대한민국").content

'대한민국의 수도는 서울이고, 캐나다의 수도는 오타와입니다.'

In [19]:
chain.invoke({"country1" : "서울", "country2" : "호주"}).content

'서울은 대한민국의 수도이고, 호주의 수도는 캔버라입니다.'

### `partial_variables` : 부분 변수 채움
`partial`을 사용하는 일반적인 용도는 함수를 부분적으로 사용하는 것입니다. </br>

이 사용 사례는 항상 공통된 방식으로 가져오고 싶은 변수가 있는 경우입니다. </br>

대표적인 예가 날짜나 시간입니다. </br>

항상 현재 날짜가 표시되기를 원하는 프롬프트가 있다고 가정해 보겠습니다. 프롬프트에 하드 코딩할 수도 없고, 다른 입력 변수와 함께 전달하는 것도 번거롭습니다. 이 경우 항상 현재 날짜를 반환하는 함수를 사용하여 프롬프트를 부분적으로 변경할 수 있으면 매우 편리합니다.

In [20]:
# 다음 코드는 오늘 날짜를 구하는 파이썬 코드입니다.
from datetime import datetime

# 오늘 날짜를 출력
datetime.now().strftime("%B %d")

'December 27'

In [21]:
# 날짜를 반환하는 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

In [26]:
prompt = PromptTemplate(
    template="오늘 날짜는 {today}입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today":get_today # dictionary 형태로 partital_variable를 전달
    },
)

prompt

PromptTemplate(input_variables=['n'], input_types={}, partial_variables={'today': <function get_today at 0x1501c62a0>}, template='오늘 날짜는 {today}입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.')

In [23]:
# prompt 생성
prompt.format(n=3)

'오늘 날짜는 December 27입니다. 오늘이 생일인 유명인 3명을 나열해 주세요. 생년월일을 표기해주세요.'

In [24]:
# chain 생성
chain = prompt | llm

In [27]:
# chain을 실행한 후 결과를 확인합니다.
print(chain.invoke(3).content)

1. 박신양 - 1954년 12월 27일
2. 에밀리아 클라크 - 1996년 12월 27일
3. 류준열 - 1986년 12월 27일


In [29]:
# chain을 실행한 후 결과를 확인합니다.
print(chain.invoke({"today" : "January 02", "n" : 3}).content)

1. Kate Bosworth - 1983년 1월 2일
2. Taye Diggs - 1971년 1월 2일
3. Christy Turlington - 1969년 1월 2일
